In [ ]:
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("./input"))

# Any results you write to the current directory are saved as output.

Підключіть необхідні бібліотеки.

In [111]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(rc={'figure.figsize':(10, 8)}); # you can change this if needed

## Data loading

Завантажте дані. Вони знаходяться у директорії input.

In [112]:
df = pd.read_csv('./input/bank-additional-full.csv', delimiter = ';')

## Data understanding

* Виведіть загальну інформацію про набір даних. Яку кількість змінних він містить? Який тип змінних? Чи багато пропущених значень?
* Виведіть описову статистику по числовим змінним. Зробіть візуалізацію розподілів найцікавіших із них. Що можна сказати про наявність помилкових значень? Чи багато нетипових значень?
* Виведіть описову статистику по категоріальним змінним. Зробіть візуалізацію розподілів найцікавіших із них.

In [ ]:
df.info()

* кількість змінних - 21.
* Тип змінних - цілі(int64), дійсні(float64) та об'екти(object).
* Пропущених значень немає усі non-null.

In [ ]:
df.head(5)

In [ ]:
# далі йде описова статистика числових змінних
df.describe().T

In [ ]:
numeric_cols = ['age', 'duration', 'campaign', 'pdays', 'previous', 'emp.var.rate', 'cons.price.idx', 'cons.conf.idx', 'euribor3m', 'nr.employed']
sns.pairplot(df[numeric_cols])
plt.show()


In [ ]:
for x in numeric_cols:
    sns.boxplot(df[x])
    plt.title(x)
    plt.show()

У змінних 'age', 'duration', 'campaign', 'pdays', 'previous' е виброси, нетипові значення. Помилкови значення duration = 0,campaign > 50 - значення більше 50 свідчать про аномалію.

Висновки:
* Для багатьох змінних є потенційні виброси або нетипові значення, але це не завжди свідчить про помилку (наприклад, літні клієнти старші за 90 років можуть бути реальними).
* Окремі змінні, такі як duration, можуть мати 0 значення, що варто вважати неінформативними для моделі.
* Високі значення в таких змінних, як campaign або previous, можуть бути аномальними, особливо якщо вони значно перевищують середні або 75-й процентиль.

категоріальні змінні

In [ ]:
# Подивимося на унікальні значення та частоти для категоріальних змінних
categorical_cols = df.select_dtypes(include=['object']).columns
df[categorical_cols].describe().T



In [ ]:
# Візуалізація розподілу для змінної 'job'
plt.figure(figsize=(10, 6))
sns.countplot(x='job', data=df)
plt.title('Розподіл за професіями')
plt.xticks(rotation=90)
plt.show()

# Візуалізація розподілу для змінної 'marital'
plt.figure(figsize=(6, 4))
sns.countplot(x='marital', data=df)
plt.title('Розподіл за сімейним станом')
plt.show()

# Візуалізація розподілу для змінної 'education'
plt.figure(figsize=(8, 5))
sns.countplot(x='education', data=df)
plt.title('Розподіл за рівнем освіти')
plt.xticks(rotation=45)
plt.show()

Основні висновки щодо розподілу категоріальних змінних job (професія), marital (сімейний стан) і education (освіта):

1. Професія (job):
* Найбільше клієнтів працюють в адмініструванні та на виробництві ("admin." і "blue-collar").
* Мало клієнтів серед студентів та безробітних.
* Маркетинг: Можна налаштувати окремі пропозиції для різних професій, адже кожна з них має різні фінансові потреби.
2. Сімейний стан (marital):
* Більшість клієнтів — одружені.
* Менше клієнтів серед неодружених і розлучених.
* Маркетинг: Одружені клієнти можуть мати більше потреб у довгострокових фінансових продуктах.
3. Освіта (education):
* Найбільше клієнтів з вищою освітою.
* Клієнтів з базовою освітою менше.
* Маркетинг: Продукти можна адаптувати під різні рівні освіти — для клієнтів з вищою освітою можуть бути цікаві складніші банківські продукти.

Загалом:
* Основна аудиторія — це одружені клієнти з вищою освітою.
* Є можливість краще орієнтувати пропозиції на різні групи клієнтів.

Загальний висновок:
Багато змінних мають категорії з великим перекосом у значеннях (наприклад, більшість клієнтів не мають кредитів, іпотек або дефолту).
Цільова змінна y має значний дисбаланс класів: більшість клієнтів (понад 88%) не підписалися на депозит.

## Data cleaning & Analyzing relationships

У подальших етапах за необхідності видаляйте (або заповнюйте певним чином) пропущені значення та/або викиди (outliers). Цей крок не є обов'язковим, проте слід пам'ятати, що деякі коефіцієнти кореляції та моделі чутливі до наявності нетипових значень у даних.

Який середній вік неодружених клієнтів?

In [ ]:
# Your code
df[df['marital']!='married']['age'].mean()

В який день тижня (ознака **day_of_week**) найчастіше дзвонили клієнтам, які відмовилися від депозиту?

In [ ]:
# Your code
day_of_week_no_deposit = df[df['y'] == 'no']['day_of_week'].value_counts()
# print(day_of_week_no_deposit)
most_frequent_day = day_of_week_no_deposit.idxmax()
most_frequent_count = day_of_week_no_deposit.max()
print(f"Найчастіше дзвонили у {most_frequent_day}: {most_frequent_count} дзвінків")


Побудуйте інфографіку за ознаками **marital** та **y**. Які висновки можна зробити?

In [ ]:
# Your code
# Створимо зведену таблицю для підрахунку кількості клієнтів за ознаками "marital" і "y"
marital_y_counts = df.groupby(['marital', 'y']).size().unstack()

# Побудуємо стовпчасту діаграму
marital_y_counts.plot(kind='bar', stacked=True, figsize=(10, 6))

# Додаємо заголовки та підписи осей
plt.title('Розподіл клієнтів за сімейним станом і підпискою на депозит')
plt.xlabel('Сімейний стан')
plt.ylabel('Кількість клієнтів')
plt.legend(title='Підписка на депозит')
plt.show()

Висновки:
* Більшість клієнтів — одружені, але навіть серед них підписка на депозит не є поширеною.
* Для ефективного маркетингу можна краще орієнтуватися на одружених клієнтів, оскільки вони складають основну частку, але також важливо шукати способи залучити клієнтів інших категорій сімейного стану.

Чи є зв'язок між наявністю кредиту (**default**) у клієнта та результатом? Побудуйте інфографіку.

In [ ]:
# Your code
plt.figure(figsize=(8, 6))
sns.countplot(x='default', hue='y', data=df)
plt.title('Зв\'язок між наявністю кредиту (default) та результатом підписки на депозит')
plt.xlabel('Наявність кредиту (default)')
plt.ylabel('Кількість клієнтів')
plt.legend(title='Підписка на депозит', loc='upper right')
plt.show()

Висновок зв'язок є
* Клієнти без кредіту найчастіше підписалися на депозит 
* Клієнти зі статусом невідомо частіше відповідали ні

Візуалізуйте інформацію про рівень освіти залежно від середнього віку. Чи є статистично значущим взаємозв'язок між ними?

In [ ]:
# Your code
plt.figure(figsize=(10, 6))
sns.barplot(x='education', y='age', data=df, estimator=np.mean)
plt.title('Середній вік клієнтів залежно від рівня освіти')
plt.xlabel('Рівень освіти')
plt.ylabel('Середній вік')
plt.xticks(rotation=45)
plt.show()

Вважаю що взаємозв'язок між ними не є статистично значущим, тому що в цілому картина одинакова для різніх видів освіти

Що можна сказати про зв'язок між тривалістю контакту (ознака **duration**) та віком клієнта? Знайдіть коефіцієнт кореляції.

In [ ]:
# Your code
correlation = df['duration'].corr(df['age'])
print(f"Коефіцієнт кореляції між duration і age: {correlation}")

Коефіцієнт кореляції практично нульовий тому можна сказати що вони ніяк незалежать один від одного.

Як пов'язані ознаки **education** та **housing**?

In [ ]:
# Your code
# Створюємо таблицю перехресних частот для education та housing
education_housing_crosstab = pd.crosstab(df['education'], df['housing'])
print(education_housing_crosstab)

In [ ]:
# Візуалізація розподілу ознак education і housing
plt.figure(figsize=(10, 6))
sns.countplot(x='education', hue='housing', data=df)
plt.title('Розподіл клієнтів за рівнем освіти і наявністю іпотеки')
plt.xlabel('Рівень освіти')
plt.ylabel('Кількість клієнтів')
plt.xticks(rotation=45)
plt.show()

Загальні висновки:
* Клієнти з вищою освітою та професійною підготовкою частіше мають іпотеку.
* Клієнти з базовою освітою мають приблизно однакову ймовірність наявності або відсутності іпотеки.
* Невідомий рівень освіти не показує чіткої тенденції, але частка клієнтів з іпотекою також присутня.

Які ще інсайти можна знайти в наявних даних? Творче завдання.

In [ ]:
# Your code
# Вибираємо економічні індикатори
economic_indicators = ['emp.var.rate', 'cons.price.idx', 'cons.conf.idx', 'euribor3m', 'nr.employed']

# Створюємо зведену таблицю середніх значень для підписаних і непідписаних клієнтів
economic_analysis = df.groupby('y')[economic_indicators].mean()

print(economic_analysis)

In [ ]:
for indicator in economic_indicators:
    plt.figure(figsize=(8, 6))
    sns.boxplot(x='y', y=indicator, data=df)
    plt.title(f'Вплив {indicator} на підписку')
    plt.xlabel('Підписка на депозит')
    plt.ylabel(indicator)
    plt.show()

Висновок
* Клієнти частіше підписуються на депозити під час економічного спаду (зниження зайнятості, нижчий рівень Euribor). Це може свідчити про те, що в кризових умовах клієнти шукають надійні фінансові інструменти для збереження своїх коштів.
* Зайнятість і ставка Euribor відіграють важливу роль у прийнятті рішень щодо підписки на депозит. Коли економічна ситуація стабільна, клієнти менше схильні вкладати кошти у депозити, і навпаки, під час нестабільності вони більше цікавляться цими продуктами.
* Індекс споживчої довіри і споживчі ціни не демонструють значного впливу на рішення клієнтів, але деякі клієнти з більшою довірою до економіки можуть бути зацікавлені в підписці на депозити.